# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [1]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

In [9]:
a="this is a " +\
    "string"

In [10]:
a

'this is a string'

In [11]:
a=("this is a " +
      "string")

In [12]:
a

'this is a string'

In [13]:
a='''
this is one line
this is another
'''

In [14]:
a

'\nthis is one line\nthis is another\n'

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [18]:
thing = pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

In [19]:
type(thing)

pandas.core.frame.DataFrame

In [20]:
thing.head(2)

id                                     address  latitude  \
0  2121978635   829 Folsom St UNIT 906, San Francisco, CA  37781429   
1    89239580  690 Market St UNIT 1705, San Francisco, CA  37788246   

   longitude  is_a_studio      ...          size  size_unit  lot_size  \
0 -122401860            0      ...         557.0       sqft       NaN   
1 -122403198            0      ...        1050.0       sqft       NaN   

  lot_size_unit  built_in_year  
0          None         2010.0  
1          None         2007.0  

[2 rows x 12 columns]

### `SELECT`

In [21]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

id  sale_price sale_price_unit
0     15165953   650000.00               $
1     80749447        1.15              $M
2     15155751   665000.00               $
3     15143887        2.10              $M
4     15117639        1.35              $M
..         ...         ...             ...
995   69819708   731000.00               $
996   15076156        5.53              $M
997  119685619   625000.00               $
998   15113584   895000.00               $
999   15068874   650000.00               $

[1000 rows x 3 columns]

### `WHERE`

In [22]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

id  sale_price
0      80749447        1.15
1      15143887        2.10
2      15117639        1.35
3      80743040        1.48
4      15134909        1.09
..          ...         ...
570  2101028916        2.45
571    15091950        1.75
572    89237766        1.16
573    15197693        1.36
574    15076156        5.53

[575 rows x 2 columns]

> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [23]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

id  sale_price
0     15165953       0.650
1     15155751       0.665
2    124852113       0.825
3     15124281       0.625
4     15071070       0.718
..         ...         ...
420  119685503       0.750
421   69819708       0.731
422  119685619       0.625
423   15113584       0.895
424   15068874       0.650

[425 rows x 2 columns]

### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [24]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

id  sale_price
0      15165953       0.650
1      15155751       0.665
2     124852113       0.825
3      15124281       0.625
4      15071070       0.718
..          ...         ...
995  2101028916       2.450
996    15091950       1.750
997    89237766       1.160
998    15197693       1.360
999    15076156       5.530

[1000 rows x 2 columns]

In [29]:
pd.io.sql.read_sql('''
SELECT id, 
        CASE WHEN sale_price_unit='$' THEN sale_price / 1000000
             WHEN sale_price_unit='$M' THEN sale_price 
        END AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

id  sale_price
0     15165953       0.650
1     15155751       0.665
2    124852113       0.825
3     15124281       0.625
4     15071070       0.718
..         ...         ...
420  119685503       0.750
421   69819708       0.731
422  119685619       0.625
423   15113584       0.895
424   15068874       0.650

[425 rows x 2 columns]

### `WITH`

In [27]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

id  sale_price
0      15165953       0.650
1      15155751       0.665
2     124852113       0.825
3      15124281       0.625
4      15071070       0.718
..          ...         ...
995  2101028916       2.450
996    15091950       1.750
997    89237766       1.160
998    15197693       1.360
999    15076156       5.530

[1000 rows x 2 columns]

### `JOIN`

In [28]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

baths  sale_price
0      1.0       0.650
1      2.0       0.665
2      NaN       0.825
3      1.0       0.625
4      1.0       0.718
..     ...         ...
995    3.0       2.450
996    NaN       1.750
997    2.5       1.160
998    2.0       1.360
999    7.5       5.530

[1000 rows x 2 columns]

### `WITH` (take 2)

In [30]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

baths  sale_price
0      1.0       0.650
1      2.0       0.665
2      NaN       0.825
3      1.0       0.625
4      1.0       0.718
..     ...         ...
995    3.0       2.450
996    NaN       1.750
997    2.5       1.160
998    2.0       1.360
999    7.5       5.530

[1000 rows x 2 columns]

### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [33]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

count
0    164
1    142
2    316
3    226
4    103
5     28
6      9
7      6
8      3
9      3

### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [46]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT baths, AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

baths  mean_sale_price
0     NaN         1.739026
1    1.00         0.987656
2    1.10         1.420000
3    1.25         1.600000
4    1.50         1.223378
..    ...              ...
16   6.50        16.000000
17   7.00         0.999000
18   7.50         5.530000
19   8.00        13.100000
20  14.00         1.000000

[21 rows x 2 columns]

Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [48]:
# TODO
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price, is_a_studio
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT 
        AVG(sale_price) 
        
    FROM baths_normalized_sale_prices
    WHERE is_a_studio=1
   
;
''', con = db)

AVG(sale_price)
0         1.464034

> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [53]:
# TODO
pd.io.sql.read_sql(
'''
SELECT a.id, b.id, a.address, b.address
    FROM properties as a
    JOIN properties as b ON b.latitude=b.latitude AND a.latitude=b.latitude
    where a.address < b.address
   
;
''', con = db)

id        id                                          address  \
0    15076699  15076711  1 Daniel Burnham Ct APT 1202, San Francisco, CA   
1    15068551  15068514    1177 California St APT 501, San Francisco, CA   
2    15068580  15068514    1177 California St APT 423, San Francisco, CA   
3    15068592  15068514    1177 California St APT 304, San Francisco, CA   
4    69819664  80730869         260 King St UNIT 1205, San Francisco, CA   
..        ...       ...                                              ...   
118  15075139  89237930            2215 California St, San Francisco, CA   
119  15078423  15078419              301 Gough St # G, San Francisco, CA   
120  63107226  67395015     3981 Alemany Blvd APT 206, San Francisco, CA   
121  67394898  67395015    8300 Oceanview Ter APT 306, San Francisco, CA   
122  69819469  69819412         250 King St UNIT 1304, San Francisco, CA   

                                             address  
0    1 Daniel Burnham Ct APT 1404, San Francisco, CA  
1      1177 California St APT 702, San Francisco, CA  
2      1177 California St APT 702, San Francisco, CA  
3      1177 California St APT 702, San Francisco, CA  
4            260 King St UNIT 817, San Francisco, CA  
..                                               ...  
118            333 Main St # IT8H, San Francisco, CA  
119            301 Gough St APT 3, San Francisco, CA  
120    8400 Oceanview Ter APT 218, San Francisco, CA  
121    8400 Oceanview Ter APT 218, San Francisco, CA  
122          250 King St UNIT 410, San Francisco, CA  

[123 rows x 4 columns]